# Tag 'Nec' means the necerssary cells

In [6]:
'''
Nec
'''

'''
PDB file to Sequence
'''
code_standard = {
    'ALA':'A', 'VAL':'V', 'PHE':'F', 'PRO':'P', 'MET':'M',
    'ILE':'I', 'LEU':'L', 'ASP':'D', 'GLU':'E', 'LYS':'K',
    'ARG':'R', 'SER':'S', 'THR':'T', 'TYR':'Y', 'HIS':'H',
    'CYS':'C', 'ASN':'N', 'GLN':'Q', 'TRP':'W', 'GLY':'G',
    #'MSE':'M',
    ' DA':'a', ' DC':'c', ' DG':'g', ' DT':'t',
    '  A':'a', '  C':'c', '  G':'g', '  T':'u',
    }

def pdbfile2seq(pdb_path):
    lines = open(pdb_path).readlines()
    seq = []
    n = 0
    lastchain = None
    for line in lines:
        if line[:4]!='ATOM':continue
        res_num=line[22:27].strip() # residue sequence number
        residue=line[17:20]
        res = code_standard.get(residue,'X')
        chain = line[21]
        if lastchain == None:
            lastchain = chain
        else:
            if chain != lastchain:
                break
        if res_num == n:
            continue
        else:
            n = res_num
            seq.append(res)
    sequence = ''.join(seq)
    return chain, sequence

def readseq(path):
    with open(path) as f:
        lines = f.read().split('\n')
    return lines[1]

In [10]:
import pickle as pkl
with open('../../data/Davis/name_to_seq.pkl', 'rb') as f:
    name_to_seq = pkl.load(f)
name_to_seq['TESK1']

'LKMNKLPSNRGNTLREVQLMNRLRHPNILRFMGVCVHQGQLHALTEYMNGGTLEXLLSSPEPLSWPVRLHLALDIARGLRYLHSKGVFHRDLTSKNCLVRREDRGFTAVVGDFGLAEKIPVYREGARKEPLAVVGSPYWMAPEVLRGELYDEKADVFAFGIVLCELIARVPADPDYLPRTEDFGLDVPAFRTLVGDDCPLPFLLLAIHCCNLEPSTRAPFTEITQHLEWILEQLPEPAPLTXTA'

In [7]:
pdb_seq, seq

('LKMNKLPSNRGNTLREVQLMNRLRHPNILRFMGVCVHQGQLHALTEYMNGGTLELLSSPEPLSWPVRLHLALDIARGLRYLHSKGVFHRDLTSKNCLVRREDRGFTAVVGDFGLAEKIPVYREGARKEPLAVVGSPYWMAPEVLRGELYDEKADVFAFGIVLCELIARVPADPDYLPRTEDFGLDVPAFRTLVGDDCPLPFLLLAIHCCNLEPSTRAPFTEITQHLEWILEQLPEPAPLTTA',
 'LKMNKLPSNRGNTLREVQLMNRLRHPNILRFMGVCVHQGQLHALTEYMNGGTLEXLLSSPEPLSWPVRLHLALDIARGLRYLHSKGVFHRDLTSKNCLVRREDRGFTAVVGDFGLAEKIPVYREGARKEPLAVVGSPYWMAPEVLRGELYDEKADVFAFGIVLCELIARVPADPDYLPRTEDFGLDVPAFRTLVGDDCPLPFLLLAIHCCNLEPSTRAPFTEITQHLEWILEQLPEPAPLTXTA')

In [11]:
# Check Sequences
import os
unmatched = []
for f in os.listdir('cut_seqs'):
    # print(f)
    if f[-3:] != 'seq':continue
    seq = readseq('cut_seqs/'+f)
    pdb_seq = pdbfile2seq('pdbs/'+f[:-4]+'.pdb')[1]
    if seq != pdb_seq:
        unmatched.append(f)
print('done')
print(unmatched)

'''There are 2 X residues in TESK1.seq which are not supported in OmegaFold'''

done
['TESK1.seq']


## New Map From Bio

In [15]:
# seq, len(seq)
import pickle as pkl
with open('omega/TESK1.pkl','rb') as f:
    o = pkl.load(f)
o['struct_node'].shape, o['struct_edge'].shape, len('LKMNKLPSNRGNTLREVQLMNRLRHPNILRFMGVCVHQGQLHALTEYMNGGTLELLSSPEPLSWPVRLHLALDIARGLRYLHSKGVFHRDLTSKNCLVRREDRGFTAVVGDFGLAEKIPVYREGARKEPLAVVGSPYWMAPEVLRGELYDEKADVFAFGIVLCELIARVPADPDYLPRTEDFGLDVPAFRTLVGDDCPLPFLLLAIHCCNLEPSTRAPFTEITQHLEWILEQLPEPAPLTTA')

((244, 384), (244, 244, 128), 242)

In [11]:
'''
Nec
'''
def calc_residue_dist(residue_one, residue_two) :
    """Returns the C-alpha distance between two residues"""
    diff_vector  = residue_one["CA"].coord - residue_two["CA"].coord
    return numpy.sqrt(numpy.sum(diff_vector * diff_vector))

def calc_dist_matrix(chain_one, chain_two) :
    """Returns a matrix of C-alpha distances between two chains"""
    answer = numpy.zeros((len(chain_one), len(chain_two)), numpy.float64)
    for row, residue_one in enumerate(chain_one) :
        for col, residue_two in enumerate(chain_two) :
            answer[row, col] = calc_residue_dist(residue_one, residue_two)
    return answer

In [4]:
import Bio.PDB
import numpy
import os
pdb_code = "1XI4"
pdb_filename = "1XI4.pdb" #not the full cage!

In [38]:
saved = []

In [39]:
'''
Nec
'''
from tqdm import tqdm
for pdb in tqdm(os.listdir('pdbs/')):
    if pdb[-3:] != 'pdb':continue
    # path = 'pdbs/' + pdb
    # con = pcmap.contactMap(path)
    # contact_list = con['data']
    # map_size = len(contact_list)
    name = pdb[:-4]
    if name in saved:
        continue
    pdb_code = name
    pdb_filename = 'pdbs/' + pdb
    structure = Bio.PDB.PDBParser().get_structure(pdb_code, pdb_filename)
    model = structure[0]
    dist_matrix = calc_dist_matrix(model["A"], model["A"])
    contact_map = dist_matrix < 5.0
    if name == 'TESK1':
        temp = numpy.zeros((244,244))
        temp[:54, :54] = contact_map[:54, :54]
        temp[:54, 55:-3] = contact_map[:54, 54:-2]
        temp[:54, -2:] = contact_map[:54, -2:]
        
        temp[55:-3, :54] = contact_map[54:-2, :54]
        temp[55:-3, 55:-3] = contact_map[54:-2, 54:-2]
        temp[55:-3, -2:] = contact_map[54:-2, -2:]
        
        temp[-2:, :54] = contact_map[-2:, :54]
        temp[-2:, 55:-3] = contact_map[-2:, 54:-2]
        temp[-2:, -2:] = contact_map[-2:, -2:]
        contact_map = temp
    assert contact_map.shape[0] == contact_map.shape[1] == len(readseq('cut_seqs/'+name+'.seq')), name
    saved.append(name)
    numpy.save('pdb_contact_map/'+name+'.npy', contact_map)
print('done')

100%|██████████| 523/523 [21:19<00:00,  2.45s/it]

done


In [26]:
import pickle as pkl
with open('omega_2/TESK1.pkl','rb') as f:
    o = pkl.load(f)
seq = readseq('cut_seqs/'+name+'.seq')
contact_map, o['struct_node'].shape, o['struct_edge'].shape, seq.index('X'), seq

(array([[ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        [ True,  True,  True, ..., False, False, False],
        ...,
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True]]),
 (244, 384),
 (244, 244, 128),
 54,
 'LKMNKLPSNRGNTLREVQLMNRLRHPNILRFMGVCVHQGQLHALTEYMNGGTLEXLLSSPEPLSWPVRLHLALDIARGLRYLHSKGVFHRDLTSKNCLVRREDRGFTAVVGDFGLAEKIPVYREGARKEPLAVVGSPYWMAPEVLRGELYDEKADVFAFGIVLCELIARVPADPDYLPRTEDFGLDVPAFRTLVGDDCPLPFLLLAIHCCNLEPSTRAPFTEITQHLEWILEQLPEPAPLTXTA')

In [40]:
numpy.argwhere(contact_map>0).shape

(2734, 2)